<a href="https://colab.research.google.com/github/bernia/MachineLearning/blob/master/irisFlowers_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Inicializacion y carga de datos

In [0]:
import numpy as np
import scipy as sc
import tensorflow as tf
import pandas as pd
import seaborn as sns
from pandas import get_dummies
from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

#Carga de datos
data = load_iris()
X = data.data
Y = data.target

## Preproceso de datos

In [3]:
# Normalizacon de las muestras - Media 0 y Des. Tipica 1.0
for i in range(len(X)):
  X[i] = (X[i] - X[i].mean())/X[i].std()

# Partición y mezcla aleatoria de los datos

# Creamos una tabla con las muestras por fila
# y las caracteristicas por columnas
data_norm_X = pd.DataFrame(X)
data_Y = pd.DataFrame(Y)

# Mezclamos los datos
indices = data_norm_X.index.tolist()
indices = np.array(indices)
np.random.shuffle(indices)
X = data_norm_X.reindex(indices)
Y = data_Y.reindex(indices)

# Conjuntos de entrenamiento y Validacion
# Pasamos la salida a formato One Hot Encoding
Y = get_dummies(Y,columns=[0])
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=.3)

# Convertimos los conjuntos a nparrays para usarlos con tensorflow
X_train = np.array(X_train).astype(np.float32)
X_test  = np.array(X_test).astype(np.float32)
Y_train = np.array(Y_train).astype(np.float32)
Y_test  = np.array(Y_test).astype(np.float32)

#Check to make sure split still has 4 features and 3 labels
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(105, 4) (105, 3)
(45, 4) (45, 3)


## Entrenamiento y test de la red

In [0]:
# Red neuronal

# Definimos los puntos de entrada de la red, para la matriz X e Y.
iX = tf.placeholder('float', shape=[None, X_train.shape[1]])
iY = tf.placeholder('float', shape=[None])

num_features = X_train.shape[1]   # Numero de caracteristicas
num_labels = Y_train.shape[1]     # Numero de etiquetas
lr = 0.01
nn = [num_features , 20 , 12 , num_labels] # Neuronas por capa

# Capa 1
W1 = tf.Variable(tf.truncated_normal([nn[0], nn[1]]), name='Weights_1')
b1 = tf.Variable(tf.zeros([nn[1]]), name='bias_1')

l1 = tf.nn.relu(tf.add(tf.matmul(iX, W1), b1))

# Capa 2
W2 = tf.Variable(tf.truncated_normal([nn[1], nn[2]]), name='Weights_2')
b2 = tf.Variable(tf.zeros([nn[2]]), name='bias_2')

l2 = tf.nn.relu(tf.add(tf.matmul(l1, W2), b2))

# Capa 3
W3 = tf.Variable(tf.truncated_normal([nn[2], nn[3]]), name='Weights_3')
b3 = tf.Variable(tf.zeros([nn[3]]), name='bias_3')

# Vector de clasificacion - Salida
pY = tf.nn.sigmoid(tf.add(tf.matmul(l2,W3),b3))[:,0]

# Evaluación de la salida
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pY, Y_train))